# IA Avanzada en Agricultura de Precisión

## Aplicaciones Prácticas con Machine Learning y LLMs

En este cuaderno exploraremos aplicaciones avanzadas de IA en agricultura, combinando análisis predictivo, sensores IoT, y LLMs para optimizar la producción agrícola.

In [ ]:
# Instalación de librerías necesarias
!pip install openai scikit-learn numpy pandas matplotlib seaborn python-dotenv -q

In [ ]:
import openai
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from openai import OpenAI
from datetime import datetime, timedelta
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

load_dotenv()

In [ ]:
# Configuración del cliente OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Capítulo 1: Análisis Predictivo en la Agricultura

### Predicción de Riesgos de Plagas con Machine Learning + LLM

In [ ]:
# Función para obtener respuestas especializadas del LLM
def obtener_analisis_agricola(prompt, modelo="gpt-4o"):
    response = client.chat.completions.create(
        model=modelo,
        messages=[
            {"role": "system", "content": "Eres un experto agrónomo especializado en agricultura de precisión y análisis de datos agrícolas."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )
    return response.choices[0].message.content

In [ ]:
# Generación de datos sintéticos para el modelo predictivo
np.random.seed(42)

# Datos de muestra: [humedad_del_suelo, temperatura, humedad_relativa, velocidad_viento, precipitacion]
n_samples = 1000
X = np.random.rand(n_samples, 5) * np.array([100, 40, 100, 30, 50])  # Escalas realistas

# Crear etiquetas basadas en condiciones lógicas
# Riesgo alto si: humedad alta (>70), temperatura moderada (20-30), viento bajo (<10)
y = ((X[:, 0] > 70) & (X[:, 1] > 20) & (X[:, 1] < 30) & (X[:, 3] < 10)).astype(int)

# Añadir algo de ruido para hacer más realista
noise_indices = np.random.choice(n_samples, size=int(0.1 * n_samples), replace=False)
y[noise_indices] = 1 - y[noise_indices]

print(f"Dataset generado: {n_samples} muestras")
print(f"Distribución de clases - Sin plagas: {np.sum(y==0)}, Con plagas: {np.sum(y==1)}")

In [ ]:
# Entrenar el modelo predictivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.3f}")
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=['Sin Plagas', 'Con Plagas']))

In [ ]:
# Análisis en tiempo real con nuevos datos de sensores
nuevos_datos = np.array([[75, 25, 85, 8, 12]])  # [humedad_suelo, temp, humedad_rel, viento, precipitacion]

# Predecir riesgo de plagas
prediccion = model.predict(nuevos_datos)[0]
probabilidad = model.predict_proba(nuevos_datos)[0]

# Interpretar resultados
if prediccion == 1:
    riesgo = f"ALTO riesgo de infestación de plagas con una probabilidad de {probabilidad[1]*100:.1f}%."
else:
    riesgo = f"BAJO riesgo de infestación de plagas con una probabilidad de {probabilidad[0]*100:.1f}%."

print(f"Condiciones actuales:")
print(f"- Humedad del suelo: {nuevos_datos[0][0]}%")
print(f"- Temperatura: {nuevos_datos[0][1]}°C")
print(f"- Humedad relativa: {nuevos_datos[0][2]}%")
print(f"- Velocidad del viento: {nuevos_datos[0][3]} km/h")
print(f"- Precipitación: {nuevos_datos[0][4]} mm")
print(f"\nRESULTADO: {riesgo}")

In [ ]:
# Generar informe completo usando LLM
datos_sensores = f"""Humedad del suelo: {nuevos_datos[0][0]}%, 
Temperatura: {nuevos_datos[0][1]}°C, 
Humedad relativa: {nuevos_datos[0][2]}%, 
Velocidad del viento: {nuevos_datos[0][3]} km/h, 
Precipitación: {nuevos_datos[0][4]} mm"""

prompt = f"""Basado en la siguiente evaluación de riesgos de plagas: {riesgo}
Y las condiciones ambientales: {datos_sensores}

Genera un informe técnico completo que incluya:
1. Análisis de las condiciones actuales
2. Explicación del nivel de riesgo
3. Recomendaciones específicas de manejo
4. Cronograma de monitoreo
5. Medidas preventivas inmediatas"""

informe = obtener_analisis_agricola(prompt)
print("=== INFORME DE ANÁLISIS PREDICTIVO ===")
print(informe)

## Capítulo 2: Técnicas de Agricultura de Precisión

### Análisis Multi-Sensor para Recomendaciones Personalizadas

In [ ]:
# Simulación de datos de sensores IoT en tiempo real
class SensorData:
    def __init__(self, zona):
        self.zona = zona
        self.timestamp = datetime.now()
        
    def generar_datos(self):
        return {
            "zona": self.zona,
            "timestamp": self.timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "soil_moisture": np.random.normal(35, 10),  # Humedad del suelo en %
            "temperature": np.random.normal(22, 5),     # Temperatura en °C
            "ph": np.random.normal(6.5, 0.5),          # pH del suelo
            "nutrient_levels": {
                "nitrogen": np.random.normal(50, 15),   # ppm
                "phosphorus": np.random.normal(30, 10), # ppm
                "potassium": np.random.normal(40, 12)   # ppm
            },
            "crop_stage": np.random.choice(["siembra", "vegetativa", "floración", "maduración"])
        }

# Generar datos para múltiples zonas
zonas = ["Norte", "Sur", "Este", "Oeste"]
datos_sensores = {}

for zona in zonas:
    sensor = SensorData(zona)
    datos_sensores[zona] = sensor.generar_datos()
    
# Mostrar datos generados
for zona, datos in datos_sensores.items():
    print(f"\n=== ZONA {zona} ===")
    print(f"Humedad del suelo: {datos['soil_moisture']:.1f}%")
    print(f"Temperatura: {datos['temperature']:.1f}°C")
    print(f"pH: {datos['ph']:.1f}")
    print(f"Nitrógeno: {datos['nutrient_levels']['nitrogen']:.0f} ppm")
    print(f"Fósforo: {datos['nutrient_levels']['phosphorus']:.0f} ppm")
    print(f"Potasio: {datos['nutrient_levels']['potassium']:.0f} ppm")
    print(f"Etapa del cultivo: {datos['crop_stage']}")

In [ ]:
# Análisis y recomendaciones personalizadas por zona
for zona, datos in datos_sensores.items():
    # Crear descripción de datos para el LLM
    data_description = f"""Zona: {zona}
    Humedad del suelo: {datos['soil_moisture']:.1f}%
    Temperatura: {datos['temperature']:.1f}°C
    pH del suelo: {datos['ph']:.1f}
    Niveles de nutrientes:
    - Nitrógeno: {datos['nutrient_levels']['nitrogen']:.0f} ppm
    - Fósforo: {datos['nutrient_levels']['phosphorus']:.0f} ppm
    - Potasio: {datos['nutrient_levels']['potassium']:.0f} ppm
    Etapa del cultivo: {datos['crop_stage']}"""
    
    prompt = f"""Analiza los siguientes datos de sensores de agricultura de precisión y proporciona 
    recomendaciones específicas y técnicas para:
    
    1. Programa de riego optimizado
    2. Plan de fertilización específico
    3. Ajustes de pH si son necesarios
    4. Cronograma de actividades según la etapa del cultivo
    5. Alertas o precauciones especiales
    
    Datos de la zona:
    {data_description}
    
    Proporciona recomendaciones técnicas precisas con cantidades específicas."""
    
    recommendations = obtener_analisis_agricola(prompt)
    print(f"\n{'='*50}")
    print(f"RECOMENDACIONES PARA ZONA {zona}")
    print(f"{'='*50}")
    print(recommendations)
    print("\n")

## Capítulo 3: Optimización de la Productividad del Suelo

### Sistema de Recomendación Inteligente de Fertilizantes

In [ ]:
# Datos de análisis de suelo para diferentes parcelas
parcelas_suelo = {
    "Parcela_A": {
        "area_ha": 25.5,
        "soil_data": {
            "pH": 5.8,
            "organic_matter": 3.2,  # porcentaje
            "nutrient_content": {
                "nitrogen": 40,  # ppm
                "phosphorus": 25,  # ppm
                "potassium": 35   # ppm
            },
            "soil_type": "franco-arcilloso",
            "drainage": "moderado"
        },
        "crop_type": "maíz"
    },
    "Parcela_B": {
        "area_ha": 18.3,
        "soil_data": {
            "pH": 7.2,
            "organic_matter": 2.8,
            "nutrient_content": {
                "nitrogen": 55,
                "phosphorus": 15,
                "potassium": 28
            },
            "soil_type": "franco-arenoso",
            "drainage": "bueno"
        },
        "crop_type": "soja"
    },
    "Parcela_C": {
        "area_ha": 32.1,
        "soil_data": {
            "pH": 6.1,
            "organic_matter": 4.1,
            "nutrient_content": {
                "nitrogen": 35,
                "phosphorus": 45,
                "potassium": 52
            },
            "soil_type": "franco",
            "drainage": "excelente"
        },
        "crop_type": "trigo"
    }
}

print("Parcelas registradas en el sistema:")
for parcela, datos in parcelas_suelo.items():
    print(f"- {parcela}: {datos['area_ha']} ha, Cultivo: {datos['crop_type']}")

In [ ]:
# Función para generar recomendaciones de fertilización específicas
def generar_plan_fertilizacion(parcela_id, datos_parcela):
    soil_data = datos_parcela['soil_data']
    area = datos_parcela['area_ha']
    cultivo = datos_parcela['crop_type']
    
    soil_description = f"""Análisis de suelo para {parcela_id}:
    - Área: {area} hectáreas
    - Cultivo: {cultivo}
    - pH del suelo: {soil_data['pH']}
    - Materia orgánica: {soil_data['organic_matter']}%
    - Tipo de suelo: {soil_data['soil_type']}
    - Drenaje: {soil_data['drainage']}
    - Nivel de nitrógeno: {soil_data['nutrient_content']['nitrogen']} ppm
    - Nivel de fósforo: {soil_data['nutrient_content']['phosphorus']} ppm
    - Nivel de potasio: {soil_data['nutrient_content']['potassium']} ppm"""
    
    prompt = f"""Como experto en fertilidad del suelo, basado en los siguientes datos, 
    diseña un plan de fertilización técnico y específico que incluya:
    
    1. Análisis detallado de las condiciones actuales del suelo
    2. Recomendaciones específicas de fertilizantes (tipos y cantidades por hectárea)
    3. Cronograma de aplicación (fechas y métodos)
    4. Correcciones de pH si son necesarias
    5. Estimación de costos por hectárea
    6. Rendimiento esperado con el plan propuesto
    7. Monitoreo y seguimiento recomendado
    
    {soil_description}
    
    Proporciona cantidades exactas y recomendaciones técnicas precisas."""
    
    return obtener_analisis_agricola(prompt)

# Generar planes para todas las parcelas
planes_fertilizacion = {}
for parcela_id, datos in parcelas_suelo.items():
    plan = generar_plan_fertilizacion(parcela_id, datos)
    planes_fertilizacion[parcela_id] = plan
    
    print(f"\n{'='*60}")
    print(f"PLAN DE FERTILIZACIÓN - {parcela_id}")
    print(f"{'='*60}")
    print(plan)
    print("\n")

## Capítulo 4: Gestión Inteligente de Mano de Obra Agrícola

### Sistema de Programación Dinámica de Tareas

In [ ]:
# Simulación de datos ambientales y operacionales en tiempo real
def obtener_datos_ambientales():
    return {
        "weather_forecast": {
            "hoy": {
                "fecha": datetime.now().strftime("%Y-%m-%d"),
                "temp_max": np.random.normal(28, 3),
                "temp_min": np.random.normal(18, 2),
                "precipitation": np.random.exponential(5),
                "wind_speed": np.random.normal(10, 3),
                "humidity": np.random.normal(65, 10)
            },
            "pronostico_3_dias": [
                {
                    "fecha": (datetime.now() + timedelta(days=i)).strftime("%Y-%m-%d"),
                    "temp_max": np.random.normal(27, 4),
                    "precipitation": np.random.exponential(8),
                    "wind_speed": np.random.normal(12, 4)
                } for i in range(1, 4)
            ]
        },
        "soil_conditions": {
            "moisture_level": np.random.normal(45, 8),
            "temperature": np.random.normal(22, 2),
            "fertility_level": np.random.choice(["bajo", "medio", "alto"], p=[0.2, 0.5, 0.3])
        },
        "operational_data": {
            "workers_available": np.random.randint(8, 15),
            "machinery_status": {
                "tractor_1": np.random.choice(["disponible", "mantenimiento"], p=[0.8, 0.2]),
                "cosechadora": np.random.choice(["disponible", "ocupada"], p=[0.7, 0.3]),
                "pulverizadora": "disponible"
            },
            "fuel_level": np.random.uniform(0.3, 1.0)
        }
    }

# Obtener datos actuales
datos_operacionales = obtener_datos_ambientales()

print("=== DATOS OPERACIONALES ACTUALES ===")
print(f"Fecha: {datos_operacionales['weather_forecast']['hoy']['fecha']}")
print(f"Temperatura: {datos_operacionales['weather_forecast']['hoy']['temp_max']:.1f}°C")
print(f"Precipitación: {datos_operacionales['weather_forecast']['hoy']['precipitation']:.1f}mm")
print(f"Humedad del suelo: {datos_operacionales['soil_conditions']['moisture_level']:.1f}%")
print(f"Trabajadores disponibles: {datos_operacionales['operational_data']['workers_available']}")
print(f"Estado de maquinaria: {datos_operacionales['operational_data']['machinery_status']}")
print(f"Nivel de combustible: {datos_operacionales['operational_data']['fuel_level']:.0%}")

In [ ]:
# Generar programación dinámica de tareas
def generar_programa_trabajo(datos):
    # Preparar descripción de datos para el LLM
    weather_today = datos['weather_forecast']['hoy']
    soil = datos['soil_conditions']
    operational = datos['operational_data']
    
    environment_description = f"""CONDICIONES OPERACIONALES ACTUALES:
    
    CLIMA HOY ({weather_today['fecha']}):
    - Temperatura máxima: {weather_today['temp_max']:.1f}°C
    - Temperatura mínima: {weather_today['temp_min']:.1f}°C
    - Precipitación pronosticada: {weather_today['precipitation']:.1f}mm
    - Velocidad del viento: {weather_today['wind_speed']:.1f} km/h
    - Humedad relativa: {weather_today['humidity']:.1f}%
    
    CONDICIONES DEL SUELO:
    - Humedad del suelo: {soil['moisture_level']:.1f}%
    - Temperatura del suelo: {soil['temperature']:.1f}°C
    - Nivel de fertilidad: {soil['fertility_level']}
    
    RECURSOS DISPONIBLES:
    - Trabajadores disponibles: {operational['workers_available']} personas
    - Estado de maquinaria: {operational['machinery_status']}
    - Nivel de combustible: {operational['fuel_level']:.0%}
    
    PRONÓSTICO PRÓXIMOS 3 DÍAS:"""
    
    for i, forecast in enumerate(datos['weather_forecast']['pronostico_3_dias']):
        environment_description += f"\n    Día {i+1} ({forecast['fecha']}): {forecast['temp_max']:.1f}°C, {forecast['precipitation']:.1f}mm lluvia"
    
    prompt = f"""{environment_description}
    
    Como experto en gestión agrícola con IA, genera un programa de trabajo dinámico y optimizado que incluya:
    
    1. PRIORIZACIÓN DE TAREAS para hoy basada en condiciones actuales
    2. ASIGNACIÓN DE PERSONAL específica por tarea
    3. CRONOGRAMA DETALLADO (horarios específicos)
    4. USO OPTIMIZADO DE MAQUINARIA disponible
    5. CONTINGENCIAS por cambios climáticos
    6. PLANIFICACIÓN PRÓXIMOS 3 DÍAS
    7. ALERTAS Y RECOMENDACIONES ESPECIALES
    
    Considera actividades como: siembra, riego, aplicación de fertilizantes, control de malezas, 
    cosecha, mantenimiento de equipos, y otras tareas agrícolas relevantes.
    
    Proporciona un plan específico, técnico y actionable."""
    
    return obtener_analisis_agricola(prompt)

# Generar programa de trabajo
programa_trabajo = generar_programa_trabajo(datos_operacionales)
print("\n" + "="*70)
print("PROGRAMA DE TRABAJO DINÁMICO - GESTIÓN IA")
print("="*70)
print(programa_trabajo)

## Capítulo 5: Sistema de Riego Inteligente en Tiempo Real

### Optimización de Recursos Hídricos con IA

In [ ]:
# Simulación de sistema de sensores de riego IoT
class SistemaRiegoInteligente:
    def __init__(self, zonas):
        self.zonas = zonas
        self.datos_historicos = self.generar_datos_historicos()
        
    def generar_datos_historicos(self):
        # Simular 30 días de datos históricos
        fechas = [(datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(30, 0, -1)]
        return {
            'fechas': fechas,
            'humedad_promedio': np.random.normal(40, 10, 30),
            'evapotranspiracion': np.random.normal(5.2, 1.5, 30),
            'precipitacion': np.random.exponential(3, 30)
        }
    
    def obtener_datos_tiempo_real(self):
        return {
            zona: {
                "moisture_level": np.random.normal(35, 12),
                "root_zone_temperature": np.random.normal(24, 3),
                "soil_conductivity": np.random.normal(0.8, 0.3),
                "potential_root_disease_risk": np.random.choice(["bajo", "moderado", "alto"], p=[0.6, 0.3, 0.1]),
                "growth_stage": np.random.choice(["vegetativa", "floración", "fructificación", "maduración"]),
                "water_requirement": np.random.choice(["bajo", "medio", "alto"], p=[0.2, 0.5, 0.3]),
                "last_irrigation": np.random.randint(1, 5)  # días desde último riego
            } for zona in self.zonas
        }

# Inicializar sistema para 4 zonas de cultivo
zonas_cultivo = ["Zona_Norte", "Zona_Sur", "Zona_Este", "Zona_Oeste"]
sistema_riego = SistemaRiegoInteligente(zonas_cultivo)

# Obtener datos en tiempo real
datos_riego = sistema_riego.obtener_datos_tiempo_real()

print("=== ESTADO ACTUAL DEL SISTEMA DE RIEGO ===")
for zona, datos in datos_riego.items():
    print(f"\n{zona}:")
    print(f"  Humedad del suelo: {datos['moisture_level']:.1f}%")
    print(f"  Temperatura zona radicular: {datos['root_zone_temperature']:.1f}°C")
    print(f"  Riesgo enfermedad radicular: {datos['potential_root_disease_risk']}")
    print(f"  Etapa de crecimiento: {datos['growth_stage']}")
    print(f"  Requerimiento hídrico: {datos['water_requirement']}")
    print(f"  Días desde último riego: {datos['last_irrigation']}")

In [ ]:
# Generar programa de riego optimizado por zona
def optimizar_programa_riego(zona_id, datos_zona, datos_clima):
    # Combinar datos de la zona con pronóstico climático
    data_description = f"""ANÁLISIS DE RIEGO - {zona_id}:
    
    CONDICIONES ACTUALES DEL SUELO:
    - Humedad del suelo: {datos_zona['moisture_level']:.1f}%
    - Temperatura zona radicular: {datos_zona['root_zone_temperature']:.1f}°C
    - Conductividad del suelo: {datos_zona['soil_conductivity']:.2f} dS/m
    - Riesgo enfermedad radicular: {datos_zona['potential_root_disease_risk']}
    - Días desde último riego: {datos_zona['last_irrigation']}
    
    ESTADO DEL CULTIVO:
    - Etapa de crecimiento: {datos_zona['growth_stage']}
    - Requerimiento hídrico: {datos_zona['water_requirement']}
    
    CONDICIONES CLIMÁTICAS:
    - Temperatura actual: {datos_clima['weather_forecast']['hoy']['temp_max']:.1f}°C
    - Precipitación esperada hoy: {datos_clima['weather_forecast']['hoy']['precipitation']:.1f}mm
    - Humedad relativa: {datos_clima['weather_forecast']['hoy']['humidity']:.1f}%
    - Velocidad del viento: {datos_clima['weather_forecast']['hoy']['wind_speed']:.1f} km/h"""
    
    prompt = f"""{data_description}
    
    Como experto en riego de precisión, genera un programa de riego optimizado que incluya:
    
    1. DECISIÓN DE RIEGO (sí/no) con justificación técnica
    2. CANTIDAD DE AGUA específica (mm o litros/m²)
    3. HORARIO ÓPTIMO de aplicación
    4. DURACIÓN Y FRECUENCIA recomendada
    5. MÉTODO DE APLICACIÓN más eficiente
    6. CONSIDERACIONES ESPECIALES (enfermedad radicular, lixiviación, etc.)
    7. PRÓXIMO RIEGO PROGRAMADO
    8. ALERTAS Y MONITOREO continuo
    
    Considera eficiencia hídrica, prevención de enfermedades, optimización nutricional 
    y condiciones climáticas actuales y pronosticadas."""
    
    return obtener_analisis_agricola(prompt)

# Generar programas de riego para todas las zonas
programas_riego = {}
for zona_id, datos_zona in datos_riego.items():
    programa = optimizar_programa_riego(zona_id, datos_zona, datos_operacionales)
    programas_riego[zona_id] = programa
    
    print(f"\n{'='*60}")
    print(f"PROGRAMA DE RIEGO OPTIMIZADO - {zona_id}")
    print(f"{'='*60}")
    print(programa)
    print("\n")

## Capítulo 6: Análisis Integrado y Dashboard de Control

### Consolidación de Datos para Toma de Decisiones

In [ ]:
# Función para generar dashboard ejecutivo integrado
def generar_dashboard_ejecutivo():
    # Consolidar todos los datos del sistema
    resumen_operacional = f"""DASHBOARD EJECUTIVO - AGRICULTURA DE PRECISIÓN IA
    Fecha: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    
    ESTADO GENERAL DE LA OPERACIÓN:
    - Total de parcelas monitoreadas: {len(parcelas_suelo)}
    - Superficie total: {sum([p['area_ha'] for p in parcelas_suelo.values()]):.1f} hectáreas
    - Zonas de riego activas: {len(zonas_cultivo)}
    - Personal disponible: {datos_operacionales['operational_data']['workers_available']} trabajadores
    
    CONDICIONES AMBIENTALES ACTUALES:
    - Temperatura: {datos_operacionales['weather_forecast']['hoy']['temp_max']:.1f}°C
    - Precipitación esperada: {datos_operacionales['weather_forecast']['hoy']['precipitation']:.1f}mm
    - Humedad promedio del suelo: {np.mean([d['moisture_level'] for d in datos_riego.values()]):.1f}%
    
    ALERTAS Y PRIORIDADES:
    - Zonas con riesgo alto de plagas: Evaluación en curso
    - Requerimientos de fertilización: {len([p for p in parcelas_suelo.values() if p['soil_data']['nutrient_content']['nitrogen'] < 45])} parcelas
    - Zonas que requieren riego inmediato: {len([d for d in datos_riego.values() if d['moisture_level'] < 30])} de {len(datos_riego)}
    
    ESTADO DE MAQUINARIA:
    - Equipos disponibles: {len([k for k, v in datos_operacionales['operational_data']['machinery_status'].items() if v == 'disponible'])}
    - Nivel de combustible: {datos_operacionales['operational_data']['fuel_level']:.0%}
    """
    
    prompt = f"""Basado en el siguiente resumen operacional de una granja con agricultura de precisión:
    
    {resumen_operacional}
    
    Genera un REPORTE EJECUTIVO INTEGRAL que incluya:
    
    1. RESUMEN EJECUTIVO (situación general y puntos clave)
    2. ANÁLISIS DE PERFORMANCE (eficiencia operacional)
    3. PRIORIDADES INMEDIATAS (próximas 24-48 horas)
    4. RECOMENDACIONES ESTRATÉGICAS (próxima semana)
    5. ANÁLISIS DE RIESGOS (climático, operacional, económico)
    6. OPORTUNIDADES DE OPTIMIZACIÓN
    7. MÉTRICAS CLAVE (KPIs sugeridos para seguimiento)
    8. PRONÓSTICO Y PLANIFICACIÓN (próximos 15 días)
    
    El reporte debe ser técnico pero accesible para la toma de decisiones gerenciales.
    Incluye recomendaciones específicas y cuantificables cuando sea posible."""
    
    return obtener_analisis_agricola(prompt)

# Generar dashboard ejecutivo
dashboard = generar_dashboard_ejecutivo()
print("\n" + "="*80)
print("DASHBOARD EJECUTIVO - AGRICULTURA DE PRECISIÓN IA")
print("="*80)
print(dashboard)

## Ejercicios Prácticos

### Implementa tu propio sistema de agricultura inteligente

In [ ]:
# EJERCICIO 1: Crear tu propio modelo predictivo
print("EJERCICIO 1: Modelo Predictivo Personalizado")
print("="*50)
print("""Tarea: Implementa un modelo predictivo para uno de estos casos:
a) Predicción de rendimiento de cultivos
b) Detección temprana de enfermedades
c) Optimización de densidad de siembra
d) Predicción de calidad de cosecha

Pasos sugeridos:
1. Define las variables de entrada relevantes
2. Genera o simula datos de entrenamiento
3. Entrena un modelo de machine learning
4. Usa LLM para interpretar y recomendar acciones
""")

# EJERCICIO 2: Sistema de alertas inteligentes
print("\nEJERCICIO 2: Sistema de Alertas Inteligentes")
print("="*50)
print("""Tarea: Desarrolla un sistema que monitoree múltiples variables 
y genere alertas automáticas cuando se detecten condiciones críticas.

Incluye:
- Umbrales dinámicos basados en cultivo y estación
- Priorización de alertas (baja, media, alta, crítica)
- Recomendaciones automáticas de acción
- Integración con LLM para explicaciones en lenguaje natural
""")

# EJERCICIO 3: Optimización multiobjetivo
print("\nEJERCICIO 3: Optimización Multiobjetivo")
print("="*50)
print("""Tarea: Crea un sistema que optimice simultáneamente:
- Maximización de rendimiento
- Minimización de costos
- Reducción de impacto ambiental
- Eficiencia en uso de recursos

Usa LLM para:
- Análizar trade-offs entre objetivos
- Sugerir estrategias balanceadas
- Comunicar decisiones complejas de forma simple
""")

## Conclusiones y Próximos Pasos

En este cuaderno hemos explorado aplicaciones avanzadas de IA en agricultura de precisión, combinando:

1. **Machine Learning** para predicción y clasificación
2. **LLMs** para interpretación y recomendaciones en lenguaje natural
3. **Sensores IoT** simulados para datos en tiempo real
4. **Sistemas integrados** para toma de decisiones holística

### Aplicaciones Implementadas:
- ✅ Predicción de riesgos de plagas
- ✅ Recomendaciones personalizadas de fertilización
- ✅ Gestión dinámica de mano de obra
- ✅ Sistemas de riego inteligente
- ✅ Dashboard ejecutivo integrado

### Próximos Desarrollos:
- Integración con APIs de clima en tiempo real
- Análisis de imágenes satelitales con IA
- Sistemas de comercialización inteligente
- Blockchain para trazabilidad
- Gemelos digitales de cultivos